# **Lab Assignment 3.2 - Create Pre-trained BERT Vectors**

### **Section 1: Base BERT Vectors**

### **Step 1: Import libraries, Mount google Drive**

In [ ]:
# Install dependencies
!pip install transformers tqdm --quiet

# Import required libraries
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Create working directory for this assignment
save_dir = "/content/drive/MyDrive/NLP_Lab3_2"
os.makedirs(save_dir, exist_ok=True)
os.chdir(save_dir)

print("Current working directory:", os.getcwd())


Mounted at /content/drive
Current working directory: /content/drive/MyDrive/NLP_Lab3_2


### **Step 2: Load Restaurant Reviews Dataset**

In [ ]:
# Load Yelp Restaurant Reviews dataset
col_name = [
    'Cust_Rating', 'Datetime', 'Review',
    'Restaurant', 'City', 'State', 'ZipCode', 'Business_Rating_Score'
]

df = pd.read_csv("/content/drive/MyDrive/NLP_Lab3_2/Restaurant_reviews_R.csv",
                 header=None, names=col_name)

# Add a ReviewID column
df = df.reset_index().rename(columns={'index': 'ReviewID'})

print("DataFrame shape:", df.shape)
print(df.head(3))


DataFrame shape: (63230, 9)
   ReviewID  Cust_Rating             Datetime  \
0         0          5.0  2008-09-29 09:33:04   
1         1          3.0  2013-12-06 23:22:26   
2         2          5.0  2008-11-16 09:44:04   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   

                    Restaurant     City State ZipCode  Business_Rating_Score  
0                    Pizza Hut  Orlando    FL   32819                    3.5  
1  La Mongerie Bakery & Bistro  Atlanta    GA   30308                    3.0  
2                    Pizza Hut  Orlando    FL   32819                    3.5  


### **Section 1: Token Length Analaysis for Base BERT**

In [ ]:
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Copy dataframe for token analysis
df_token_length = df.copy()

# Compute token ids for each review
%%time
df_token_length['token_ids'] = df_token_length['Review'].apply(
    lambda x: tokenizer.encode(x, truncation=False, add_special_tokens=True)
)

# Convert token ids back to tokens and count
df_token_length['token_count'] = df_token_length['token_ids'].apply(len)

# Identify reviews longer than 512 tokens
long_reviews = df_token_length[df_token_length['token_count'] > 512].sort_values(
    by='token_count', ascending=False
)

print(f"Number of reviews exceeding 512 tokens: {len(long_reviews)}")
print(long_reviews[['ReviewID', 'token_count']].head(10))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
UsageError: Line magic function `%%time` not found.


### **Section 1: Create Base BERT Embeddings**

In [ ]:
from transformers import BertModel
import gc

# Load model
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print("Using device:", device)

def generate_embeddings_safe(texts, batch_size=8):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size].tolist()
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            add_special_tokens=True,
            return_tensors="pt",
            max_length=512,
            return_attention_mask=True
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        batch_emb = outputs.pooler_output.detach().cpu().numpy()
        embeddings.append(batch_emb)

        # Free memory after each batch
        del inputs, outputs, batch_emb
        torch.cuda.empty_cache()
        gc.collect()

        # Save partial progress every 5000 reviews
        if (i + batch_size) % 5000 == 0:
            temp_df = pd.DataFrame(np.concatenate(embeddings, axis=0))
            temp_df.to_pickle(f"{save_dir}/temp_bert_{i+batch_size}.pkl")
            print(f"Saved checkpoint up to review {i+batch_size}")

    embeddings = np.concatenate(embeddings, axis=0)
    return embeddings

# Run safely
embeddings = generate_embeddings_safe(df['Review'], batch_size=8)

embeddings_df = pd.DataFrame(embeddings)
print("Embeddings complete:", embeddings_df.shape)
print(embeddings_df.head(3))

Using device: cuda


  8%|▊         | 625/7904 [05:03<58:11,  2.08it/s]

Saved checkpoint up to review 5000


 16%|█▌        | 1250/7904 [09:54<54:48,  2.02it/s]

Saved checkpoint up to review 10000


...

Saved checkpoint up to review 55000


 95%|█████████▍| 7500/7904 [59:00<04:39,  1.45it/s]

Saved checkpoint up to review 60000


100%|██████████| 7904/7904 [1:02:14<00:00,  2.12it/s]

Embeddings complete: (63230, 768)
        0         1         2         3         4         5         6    \
0 -0.667206 -0.501273 -0.979122  0.697638  0.878286 -0.260769  0.435067   
1 -0.588723 -0.528098 -0.980368  0.500643  0.769150 -0.161804  0.187090   
2 -0.854242 -0.446652 -0.946223  0.712074  0.800056 -0.101469  0.796103   

        7         8         9    ...       758       759       760       761  \
0  0.235805 -0.884992 -0.999859  ...  0.436511 -0.130892  0.986229  0.684380   
1  0.301930 -0.934062 -0.999857  ...  0.433274 -0.519408  0.990394  0.579297   
2  0.250824 -0.895294 -0.999977  ...  0.681200 -0.143508  0.990260  0.773459   

        762       763       764       765       766       767  
0  0.290103  0.006722  0.535985 -0.876303 -0.659951  0.820790  
1  0.328788  0.016306  0.629514 -0.912522 -0.615028  0.614334  
2 -0.183299 -0.043778  0.749857 -0.902692 -0.738916  0.907490  

[3 rows x 768 columns]


In [ ]:


# Merge original metadata with embeddings
df_basebert = pd.concat([df.reset_index(drop=True), embeddings_df], axis=1)

# Confirm merged structure
print("Merged DataFrame shape:", df_basebert.shape)
print(df_basebert.head(3))

# Save both CSV and Pickle versions
df_basebert.to_csv(f"{save_dir}/restaurant_reviews_basebert.csv", index=False)
df_basebert.to_pickle(f"{save_dir}/restaurant_reviews_basebert.pkl")

print("✅ Base BERT embeddings saved successfully!")
print("📁 Files stored in:", save_dir)


Merged DataFrame shape: (63230, 777)
   ReviewID  Cust_Rating             Datetime  \
0         0          5.0  2008-09-29 09:33:04   
1         1          3.0  2013-12-06 23:22:26   
2         2          5.0  2008-11-16 09:44:04   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   

                    Restaurant     City State ZipCode  Business_Rating_Score  \
0                    Pizza Hut  Orlando    FL   32819                    3.5   
1  La Mongerie Bakery & Bistro  Atlanta    GA   30308                    3.0   
2                    Pizza Hut  Orlando    FL   32819                    3.5   

          0  ...       758       759       760       761       762       763  \
0 -0.667206  ...  0.436511 -0.130892  0.986229  0.684380  0.290103  0.006722   
1 -0.588723  ...  0.433274 -0.519408  0.990394  0.579297  0.

### **Create the Base BERT DataFrame + Show first 5 rows**

In [ ]:
# Combine metadata with the 768-dim Base BERT embeddings
df_basebert = pd.concat([df.reset_index(drop=True), embeddings_df], axis=1)

# Confirm structure and preview first 5 reviews
print("✅ Final Base BERT DataFrame created successfully!")
print("DataFrame shape:", df_basebert.shape)
print("\n--- Preview of first 5 reviews with BERT vectors ---")
print(df_basebert.head(5))

# Optional: re-save the finalized file (to ensure metadata + vectors are together)
df_basebert.to_csv(f"{save_dir}/restaurant_reviews_basebert_final.csv", index=False)
df_basebert.to_pickle(f"{save_dir}/restaurant_reviews_basebert_final.pkl")

print("\n📁 Final Base BERT files saved in:", save_dir)


✅ Final Base BERT DataFrame created successfully!
DataFrame shape: (63230, 777)

--- Preview of first 5 reviews with BERT vectors ---
   ReviewID  Cust_Rating             Datetime  \
0         0          5.0  2008-09-29 09:33:04   
1         1          3.0  2013-12-06 23:22:26   
2         2          5.0  2008-11-16 09:44:04   
3         3          5.0  2016-04-11 17:00:34   
4         4          4.0  2015-05-26 06:47:56   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   
3  Always a great place to taste some tea with fr...   
4  Delish! Stopped in on a random Sunday and so g...   

                    Restaurant          City State ZipCode  \
0                    Pizza Hut       Orlando    FL   32819   
1  La Mongerie Bakery & Bistro       Atlanta    GA   30308   
2                    Pizza Hut       Orlando    

## **Section 2: Longform Vectors**

### **Step 1: Load Model and Tokenizer**

In [ ]:
from transformers import LongformerTokenizer, LongformerModel
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load Longformer tokenizer and model
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model.to(device)
model.eval()


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Using device: cuda


LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0-11): 12 x LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
    

### **Step 2: Generate Longform Embeddings**

In [ ]:
def generate_embeddings_longformer(texts, batch_size=8):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size].tolist()
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            add_special_tokens=True,
            return_tensors="pt",
            max_length=4096,
            return_attention_mask=True
        )

        # Add global attention on the first token ([CLS])
        global_attention_mask = torch.zeros_like(inputs['input_ids'])
        global_attention_mask[:, 0] = 1

        # ✅ Move all tensors (including global_attention_mask) to GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}
        global_attention_mask = global_attention_mask.to(device)

        with torch.no_grad():
            outputs = model(**inputs, global_attention_mask=global_attention_mask)

        batch_emb = outputs.pooler_output.detach().cpu().numpy()
        embeddings.append(batch_emb)

        # Memory cleanup
        del inputs, outputs, batch_emb, global_attention_mask
        torch.cuda.empty_cache()

    embeddings = np.concatenate(embeddings, axis=0)
    return embeddings



### **Step 3: Run Longform on my Data**

In [ ]:
%%time
embeddings_longformer = generate_embeddings_longformer(df['Review'], batch_size=8)

embeddings_longformer_df = pd.DataFrame(embeddings_longformer)
print("Embeddings shape:", embeddings_longformer_df.shape)
print(embeddings_longformer_df.head(3))


100%|██████████| 7904/7904 [1:04:17<00:00,  2.05it/s]

Embeddings shape: (63230, 768)
        0         1         2         3         4         5         6    \
0  0.184318 -0.246028  0.058935  0.008367  0.322522 -0.214286 -0.489915   
1  0.182557 -0.322504  0.086091 -0.002979  0.373754 -0.207752 -0.509066   
2  0.188996 -0.306773  0.068258  0.006348  0.321982 -0.206972 -0.493740   

        7         8         9    ...       758       759       760       761  \
0 -0.408545 -0.100367 -0.241540  ... -0.201913  0.001204  0.177015  0.098792   
1 -0.397304 -0.139065 -0.224550  ... -0.257217  0.080177  0.181387  0.130525   
2 -0.371401 -0.125890 -0.220582  ... -0.248826  0.000466  0.168063  0.109993   

        762       763       764       765       766       767  
0 -0.229145  0.033996 -0.262646 -0.081905  0.160031 -0.007477  
1 -0.172328  0.048528 -0.245074 -0.074264  0.195651  0.022976  
2 -0.191301  0.018323 -0.248086 -0.069785  0.174766  0.023799  

[3 rows x 768 columns]
CPU times: user 1h 2min 17s, sys: 1min 24s, total: 1h 3min 42s
Wall

### **Step 4: Combine, Save, and Preview Longformer Embeddings**

In [ ]:
# Step 4: Combine Longformer embeddings with original metadata
save_dir = "/content/drive/MyDrive/NLP_Lab3_2"

# Combine review metadata with Longformer vectors
df_longformer = pd.concat([df.reset_index(drop=True), embeddings_longformer_df], axis=1)

print("\n Final Longformer DataFrame created successfully!")
print("DataFrame shape:", df_longformer.shape)
print("\n--- Preview of first 5 reviews with Longformer vectors ---")
print(df_longformer.head(5))

# Save DataFrame to Google Drive (CSV + Pickle for future reuse)
df_longformer.to_csv(f"{save_dir}/restaurant_reviews_longformer_final.csv", index=False)
df_longformer.to_pickle(f"{save_dir}/restaurant_reviews_longformer_final.pkl")

print(f"\n Final Longformer files saved in: {save_dir}")




 Final Longformer DataFrame created successfully!
DataFrame shape: (63230, 777)

--- Preview of first 5 reviews with Longformer vectors ---
   ReviewID  Cust_Rating             Datetime  \
0         0          5.0  2008-09-29 09:33:04   
1         1          3.0  2013-12-06 23:22:26   
2         2          5.0  2008-11-16 09:44:04   
3         3          5.0  2016-04-11 17:00:34   
4         4          4.0  2015-05-26 06:47:56   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   
3  Always a great place to taste some tea with fr...   
4  Delish! Stopped in on a random Sunday and so g...   

                    Restaurant          City State ZipCode  \
0                    Pizza Hut       Orlando    FL   32819   
1  La Mongerie Bakery & Bistro       Atlanta    GA   30308   
2                    Pizza Hut       Orla

## **Section 3: Chunked Base BERT Vectors**

### **Step 1: Load Model and set up GPU**

In [ ]:
from transformers import BertTokenizer, BertModel
import torch, numpy as np, pandas as pd
from tqdm import tqdm

# Load base BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Using device:", device)


Using device: cuda


### **Step 2: Generate embeddings with chunking**

In [ ]:
def generate_chunked_bert_embeddings_safe(texts, batch_size=4, save_interval=1000):
    embeddings = []
    save_dir = "/content/drive/MyDrive/NLP_Lab3_2"

    for idx, text in enumerate(tqdm(texts, total=len(texts))):
        try:
            # Tokenize and chunk
            tokens = tokenizer.encode(text, add_special_tokens=True)
            chunks = [tokens[i:i+512] for i in range(0, len(tokens), 512)]

            chunk_embeddings = []
            for chunk in chunks:
                inputs = torch.tensor(chunk).unsqueeze(0).to(device)
                with torch.no_grad():
                    outputs = model(inputs)
                    emb = outputs.pooler_output.cpu().numpy()
                    chunk_embeddings.append(emb)

            # Average all chunk embeddings
            avg_embedding = np.mean(chunk_embeddings, axis=0)
            embeddings.append(avg_embedding)

            # Save every N reviews
            if (idx + 1) % save_interval == 0:
                temp_df = pd.DataFrame(np.vstack(embeddings))
                temp_df.to_pickle(f"{save_dir}/temp_chunkedbert_up_to_{idx+1}.pkl")
                print(f" Saved checkpoint → {idx+1} reviews processed.")

            # Free memory
            del inputs, outputs, chunk_embeddings
            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(f" Error on review {idx}: {e}")

    return np.vstack(embeddings)


### **Step 3: Gnerate Embeddings**

In [ ]:
%%time
sample_df = df.head(200)  # use first 200 reviews for demo

embeddings_chunked = generate_chunked_bert_embeddings_safe(sample_df['Review'], batch_size=4, save_interval=200)
embeddings_chunked_df = pd.DataFrame(embeddings_chunked)

print("Embeddings shape:", embeddings_chunked_df.shape)
print(embeddings_chunked_df.head(3))


100%|█████████▉| 199/200 [01:10<00:00,  2.87it/s]

 Saved checkpoint → 200 reviews processed.


100%|██████████| 200/200 [01:10<00:00,  2.82it/s]

Embeddings shape: (200, 768)
        0         1         2         3         4         5         6    \
0 -0.667206 -0.501273 -0.979122  0.697638  0.878286 -0.260769  0.435068   
1 -0.588723 -0.528098 -0.980368  0.500643  0.769150 -0.161804  0.187091   
2 -0.854242 -0.446652 -0.946223  0.712074  0.800055 -0.101469  0.796103   

        7         8         9    ...       758       759       760       761  \
0  0.235805 -0.884992 -0.999859  ...  0.436511 -0.130892  0.986229  0.684380   
1  0.301930 -0.934062 -0.999857  ...  0.433274 -0.519407  0.990394  0.579298   
2  0.250824 -0.895293 -0.999977  ...  0.681200 -0.143508  0.990260  0.773458   

        762       763       764       765       766       767  
0  0.290102  0.006722  0.535985 -0.876303 -0.659951  0.820790  
1  0.328789  0.016306  0.629514 -0.912522 -0.615028  0.614334  
2 -0.183298 -0.043778  0.749857 -0.902692 -0.738916  0.907490  

[3 rows x 768 columns]
CPU times: user 1min 9s, sys: 348 ms, total: 1min 10s
Wall time: 1min

### **Step 4: Combine and Save**

In [ ]:
save_dir = "/content/drive/MyDrive/NLP_Lab3_2"

df_bert_chunked = pd.concat([sample_df.reset_index(drop=True), embeddings_chunked_df], axis=1)

print("\n Final Chunked BERT DataFrame created successfully!")
print("DataFrame shape:", df_bert_chunked.shape)
print("\n--- Preview of first 5 reviews with Chunked BERT vectors ---")
print(df_bert_chunked.head(5))

df_bert_chunked.to_csv(f"{save_dir}/restaurant_reviews_chunkedbert_final.csv", index=False)
df_bert_chunked.to_pickle(f"{save_dir}/restaurant_reviews_chunkedbert_final.pkl")

print(f"\n Final Chunked BERT files saved in: {save_dir}")




 Final Chunked BERT DataFrame created successfully!
DataFrame shape: (200, 777)

--- Preview of first 5 reviews with Chunked BERT vectors ---
   ReviewID  Cust_Rating             Datetime  \
0         0          5.0  2008-09-29 09:33:04   
1         1          3.0  2013-12-06 23:22:26   
2         2          5.0  2008-11-16 09:44:04   
3         3          5.0  2016-04-11 17:00:34   
4         4          4.0  2015-05-26 06:47:56   

                                              Review  \
0  its the best pizza hut i'v found in florida al...   
1  This place is an interesting combo. The chef, ...   
2  Pizza Hut is great! You get huge pizzas for yo...   
3  Always a great place to taste some tea with fr...   
4  Delish! Stopped in on a random Sunday and so g...   

                    Restaurant          City State ZipCode  \
0                    Pizza Hut       Orlando    FL   32819   
1  La Mongerie Bakery & Bistro       Atlanta    GA   30308   
2                    Pizza Hut       Or

## **Section 4: Document-level (Restaurant- level) Embeddings**

### **Step 1: Reload the DataFrame**

In [ ]:
import pandas as pd

df_bert_chunked = pd.read_pickle("/content/drive/MyDrive/NLP_Lab3_2/restaurant_reviews_chunkedbert_final.pkl")
print("Loaded DataFrame shape:", df_bert_chunked.shape)
df_bert_chunked.head(2)


Loaded DataFrame shape: (200, 777)


,ReviewID,Cust_Rating,Datetime,Review,Restaurant,City,State,ZipCode,Business_Rating_Score,0,...,758,759,760,761,762,763,764,765,766,767
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,-0.667206,...,0.436511,-0.130892,0.986229,0.684380,0.290102,0.006722,0.535985,-0.876303,-0.659951,0.820790
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,-0.588723,...,0.433274,-0.519407,0.990394,0.579298,0.328789,0.016306,0.629514,-0.912522,-0.615028,0.614334


### **Step 2: Create Restaurant-level (Document) Embeddings**

In [ ]:
import numpy as np

# Identify embedding columns (0–767)
embedding_cols = [col for col in df_bert_chunked.columns if isinstance(col, int)]

# Group by Restaurant
restaurant_grouped = (
    df_bert_chunked.groupby('Restaurant')[embedding_cols]
    .mean()
    .reset_index()
)

print("Aggregated restaurant-level embeddings shape:", restaurant_grouped.shape)
restaurant_grouped.head(3)


Aggregated restaurant-level embeddings shape: (77, 769)


,Restaurant,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,3 Monkeys Pub & Grill,-0.478768,-0.523377,-0.983099,0.364347,0.880395,-0.171815,-0.091006,0.403187,-0.863594,...,0.718864,-0.161595,0.958756,0.471497,-0.120284,-0.501253,0.734479,-0.869899,-0.624150,0.610876
1,An Tobar Irish Pub,-0.634955,-0.347773,-0.953964,0.557748,0.863352,-0.150394,0.339287,0.180662,-0.808568,...,0.518483,-0.148512,0.964724,0.632467,0.376790,0.111901,0.633498,-0.871731,-0.594827,0.827853
2,Applebee's,-0.678249,-0.385936,-0.961169,0.603945,0.858128,-0.150649,0.449471,0.124687,-0.862797,...,0.589258,-0.185668,0.978195,0.679195,0.066519,0.058600,0.688033,-0.920040,-0.560204,0.853529


### **Step 3: Add MetaData Columns**

In [ ]:
# Aggregate metadata (average rating, first city/state)
metadata_cols = df_bert_chunked.groupby('Restaurant')[['City', 'State', 'Business_Rating_Score']].agg({
    'City': 'first',
    'State': 'first',
    'Business_Rating_Score': 'mean'
}).reset_index()

# Merge metadata + embeddings
df_docbert = pd.merge(metadata_cols, restaurant_grouped, on='Restaurant')

print("\n Final Document-BERT DataFrame created successfully!")
print("DataFrame shape:", df_docbert.shape)
print("\n--- Preview of first 5 restaurants with Document BERT vectors ---")
print(df_docbert.head(5))



 Final Document-BERT DataFrame created successfully!
DataFrame shape: (77, 772)

--- Preview of first 5 restaurants with Document BERT vectors ---
              Restaurant       City State  Business_Rating_Score         0  \
0  3 Monkeys Pub & Grill  Vancouver    WA                    3.0 -0.478768   
1     An Tobar Irish Pub   Maitland    FL                    3.0 -0.634955   
2             Applebee's    Atlanta    GA                    2.0 -0.678249   
3                 Boloco     Boston    MA                    3.5 -0.642120   
4      Bruegger's Bagels   Columbus    OH                    4.0 -0.793676   

          1         2         3         4         5  ...       758       759  \
0 -0.523377 -0.983099  0.364347  0.880395 -0.171815  ...  0.718864 -0.161595   
1 -0.347773 -0.953964  0.557748  0.863352 -0.150394  ...  0.518483 -0.148512   
2 -0.385936 -0.961169  0.603945  0.858128 -0.150649  ...  0.589258 -0.185668   
3 -0.379594 -0.869369  0.525902  0.837950 -0.200239  ...  0.533

### **Step 4: Save to Drive**

In [ ]:
save_dir = "/content/drive/MyDrive/NLP_Lab3_2"

df_docbert.to_csv(f"{save_dir}/restaurant_docbert_final.csv", index=False)
df_docbert.to_pickle(f"{save_dir}/restaurant_docbert_final.pkl")

print(f"\n💾 Final Document-BERT files saved in: {save_dir}")



💾 Final Document-BERT files saved in: /content/drive/MyDrive/NLP_Lab3_2
